In [ ]:
%store -r
import time
import os,json, random, string
import requests, operator
from dkube.sdk import *
from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeModelmonitor
from dkube.sdk.rsrcs.operator import DkubeCluster
from dkube.sdk.rsrcs.modelmonitor import DatasetClass,ModelType,DriftAlgo, DataType
from dkube.sdk.rsrcs.modelmonitor import DatasetFormat,DkubeModelmonitorAlert, TimeZone

In [ ]:
MONITOR_NAME = cld_config['MONITOR_NAME']
INPUT_TRAIN_TYPE = cld_config['INPUT_TRAIN_TYPE']
SERVING_DKUBE_USERNAME = cld_config['SERVING_DKUBE_USERNAME']
SERVING_DKUBE_TOKEN = cld_config['SERVING_DKUBE_TOKEN']
SERVING_DKUBE_URL = cld_config['SERVING_DKUBE_URL']
DKUBE_BASE_DATASET = cld_config['DKUBE_BASE_DATASET']
MODEL_NAME = cld_config['MODEL_NAME']
RUN_FREQUENCY = cld_config['RUN_FREQUENCY']
LIVE_DATASET = cld_config['LIVE_DATASET']

MONITORING_DKUBE_USERNAME = cld_config["MONITORING_DKUBE_USERNAME"]
MONITORING_DKUBE_TOKEN = cld_config["MONITORING_DKUBE_TOKEN"]
MONITORING_DKUBE_URL = cld_config["MONITORING_DKUBE_URL"]
SERVING_DKUBE_CLUSTER_NAME = cld_config["SERVING_DKUBE_CLUSTER_NAME"]

In [ ]:
serving_api = DkubeApi(URL=SERVING_DKUBE_URL,token=SERVING_DKUBE_TOKEN)

### Importing deployment

In [ ]:
def wait_for_deployment_running(deployment_id):
    status = None
    inference_url, inference = None, None
    while True:
        data = serving_api.get_deployment(deployment_id)
        status = data.data.inferenceservice_deployment.parameters.generated.status.state
        inference = data.data.inferenceservice_deployment.parameters.inference
        inference_url = data.data.inferenceservice_deployment.parameters.generated.details.serving.servingurl
        if status == "RUNNING":
            break
        print("waiting for deployment to be running")
        time.sleep(serving_api.wait_interval)
    return inference, inference_url

def get_dataset_version(username, dataset_name, version):
    dataset_versions = monitoring_api.get_dataset_versions(username, dataset_name)
    versions = []
    for each_version in dataset_versions:
        if each_version["version"]["name"] == version:
            uuid = each_version["version"]["uuid"]
            return f"{version}:{uuid}"
        else:
            versions.append(each_version["version"]["name"])
    return f"dataset version {version} not found, available version are {versions}"

In [ ]:
while True:
    SERVING_DEPLOYMENT_ID = serving_api.get_deployment_id(name=MONITOR_NAME)
    if SERVING_DEPLOYMENT_ID:
        break
    print("waiting for deployment to come up")
    time.sleep(serving_api.wait_interval)
inference, INFERENCE_URL = wait_for_deployment_running(SERVING_DEPLOYMENT_ID)
if not inference.enable_logs:
    print("Enabling logs")
    serving = DkubeServing(user=SERVING_DKUBE_USERNAME, name=MONITOR_NAME)
    serving.update_enable_logs(enable_logs=True)
    serving_api.update_inference(serving)
print("Inference is up at URL: ", INFERENCE_URL)
cld_config['INFERENCE_URL'] = INFERENCE_URL
cld_config['SERVING_DEPLOYMENT_ID'] = SERVING_DEPLOYMENT_ID
%store cld_config

## Checking for seperate monitoring cluster and adding cluster and deployment to the monitoring cluster.

In [ ]:
if MONITORING_DKUBE_URL:
    monitoring_api = DkubeApi(URL=MONITORING_DKUBE_URL,token=MONITORING_DKUBE_TOKEN)
    DKUBEUSERNAME = MONITORING_DKUBE_USERNAME
    ## Checking if the cluster exists
    cluster_exists = False
    if SERVING_DKUBE_CLUSTER_NAME:
        for each_cluster in monitoring_api.get_clusters()["data"]:
            if each_cluster["name"] == SERVING_DKUBE_CLUSTER_NAME:
                cluster_exists = True
                print(f"Cluster {SERVING_DKUBE_CLUSTER_NAME} already exists")
                break
        if not cluster_exists:
            msg = f'''
            Cluster {SERVING_DKUBE_CLUSTER_NAME} not found on monitoring cluster,
            Kindly verify the cluster name.
            '''
            raise Exception(msg)
    else:
        ## Generating serving cluster name if not provide
        ## A cluster with the generated name would be added
        res = ''.join(random.choices(string.ascii_lowercase + string.digits, k=4))
        SERVING_DKUBE_CLUSTER_NAME = f"{SERVING_DKUBE_USERNAME}-{res}"
        print(f"Attempt to create cluster with name {SERVING_DKUBE_CLUSTER_NAME} on monitoring setup will be taken")
        ## checking if the user have operator permissions
        op_permission = False
        if "operator" in monitoring_api.validate_token()["role"]:
            op_permission = True
        if op_permission:
        ### adding cluster
            pcluster = DkubeCluster(name=SERVING_DKUBE_CLUSTER_NAME)
            pcluster.update_kind("dkube-remote")
            pcluster.update_class("monitoring")
            pcluster.update_authtype("jwt")
            pcluster.update_url(url=SERVING_DKUBE_URL)
            pcluster.update_jwt_details(jwt_token=SERVING_DKUBE_TOKEN)
            try:
                monitoring_api.configure_clusters(pcluster.cluster)
                print("Cluster added")
            except Exception as e:
                print(e)
        else:
            msg = f'''
            User {MONITORING_DKUBE_USERNAME} doesn't have operator permissions,
            Ask the operator to add cluster first.
            '''
            raise Exception(msg)
    ## Importing deployment
    try:
        DEPLOYMENT_ID = monitoring_api.import_deployment(name=MONITOR_NAME,
                                                         cluster=SERVING_DKUBE_CLUSTER_NAME,
                                                         namespace=SERVING_DKUBE_USERNAME)
        print("Deployment Imported")
    except Exception as e:
        print(e)
        
else:
    monitoring_api = serving_api
    DKUBEUSERNAME = SERVING_DKUBE_USERNAME
    DEPLOYMENT_ID = SERVING_DEPLOYMENT_ID

In [ ]:
text_file = open("transform-data.py", "r")
#read whole file to a string
script = text_file.read()
#close file
text_file.close()

with open('thresholds.json') as f:
    thresholds = json.load(f)

In [ ]:
training_data = f'{DKUBEUSERNAME}:{DKUBE_BASE_DATASET}'
train_data_version = get_dataset_version(DKUBEUSERNAME,
                                            DKUBE_BASE_DATASET, "v1")
labelled_data = f"{DKUBEUSERNAME}:{LIVE_DATASET}"
predict_data_format = "cloudeventlogs"

### Model Monitor

In [ ]:
mm=DkubeModelmonitor(deployemnt_id = DEPLOYMENT_ID)

In [ ]:
mm.update_modelmonitor_basics(model_type=ModelType.Regression.value, 
                               input_data_type=DataType.Tabular.value,
                               data_timezone=TimeZone.UTC.value)

In [ ]:
mm.add_thresholds(thresholds=thresholds)

## Health Monitoring

In [ ]:
mm.update_deployment_monitoring_details(enabled=True, frequency=1)

## Add Drift monitoring details

In [ ]:
mm.update_drift_monitoring_details(enabled=True,frequency=5,algorithm='auto')

### Add Train, Prediction, and Labelled Datasets

In [ ]:
mm.add_datasources(data_class=str(DatasetClass.Train),name=training_data,
                   data_format="tabular",
                   version=train_data_version,transformer_script = script)
if MONITORING_DKUBE_URL:
    mm.add_datasources(data_class=str(DatasetClass.Predict), name=labelled_data,
                       data_format=predict_data_format, s3_subpath=SERVING_DEPLOYMENT_ID)
else:
    mm.add_datasources(data_class=str(DatasetClass.Predict),
                       data_format=predict_data_format,)

mm.add_datasources(data_class=str(DatasetClass.Labelled),name=labelled_data,
                   data_format="tabular", s3_subpath=SERVING_DEPLOYMENT_ID + "/livedata",
                   predict_col="charges",groundtruth_col="GT_target",timestamp_col="timestamp")

### Performance Monitoring

In [ ]:
mm.update_performance_monitoring_details(enabled=True,source_type="labelled_data",frequency=5)

### Create Model monitor

In [ ]:
monitoring_api.modelmonitor_create(mm,wait_for_completion=True)

### Extracting id of the Model Monitor

In [ ]:
# The function below can be used to fetch model monitor ID by name.
# The monitor id will be same as deployment id.
# id = monitoring_api.modelmonitor_get_id(MONITOR_NAME)
id = DEPLOYMENT_ID
print(id)

### Schema update

In [ ]:
monitoring_api.modelmonitor_update_schema(id,label='charges',schema_class='continuous',schema_type="prediction_output",selected=False)
monitoring_api.modelmonitor_update_schema(id,label='unique_id',schema_class='continuous',schema_type="row_id",selected=False)
monitoring_api.modelmonitor_update_schema(id,label='timestamp',schema_class='continuous',schema_type="timestamp",selected=False)

## age and bmi to continuous
monitoring_api.modelmonitor_update_schema(id,label='age',schema_class='continuous',schema_type='input_feature',selected=True)
monitoring_api.modelmonitor_update_schema(id,label='bmi',schema_class='continuous',schema_type='input_feature',selected=True)

## select these features
monitoring_api.modelmonitor_update_schema(id,label='sex',schema_type='input_feature',schema_class='categorical',selected=True)
monitoring_api.modelmonitor_update_schema(id,label='children',schema_type='input_feature',schema_class='categorical',selected=True)
monitoring_api.modelmonitor_update_schema(id,label='smoker',schema_type='input_feature',schema_class='categorical',selected=True)
monitoring_api.modelmonitor_update_schema(id,label='region',schema_type='input_feature',schema_class='categorical',selected=True)

### Add alerts

#### Deployment Health Alert

In [ ]:
alert = DkubeModelmonitorAlert(name='latency_alert', alert_class = 'deployment_health')
alert.add_alert_condition(metric='latency_avg',threshold=300, op=operator.gt)
monitoring_api.modelmonitor_add_alert(id,alert)

#### Feature Alert

In [ ]:
alert = DkubeModelmonitorAlert(name='age_alert', alert_class = 'feature_drift')
alert.add_alert_condition(feature='age',threshold=0.1, op=operator.gt)
monitoring_api.modelmonitor_add_alert(id,alert)

#### Performance Alert

In [ ]:
alert = DkubeModelmonitorAlert(name='mae_alert', alert_class = 'performance_decay')
alert.add_alert_condition(metric='mae',threshold=2000, op=operator.gt)
monitoring_api.modelmonitor_add_alert(id,alert)

### Start the model monitor

In [ ]:
monitoring_api.modelmonitor_start(id)

### Retraining / Rebaselining Model Monitor

### Cleanup

In [ ]:
CLEANUP = False
if CLEANUP:
    from time import sleep
    RETRIES = 4
    while RETRIES:
        mm = monitoring_api.modelmonitor_get(id)
        if mm["status"] and mm["status"]["state"].lower() != "active":
            break
        elif mm["status"] and mm["status"]["state"].lower() == "active":
            monitoring_api.modelmonitor_stop(id)
        RETRIES -= 1
        sleep(5)
    else:
        raise TimeoutError("modelmonitor failed to stopped")
    monitoring_api.modelmonitor_delete(id)